In [39]:
import pandas as pd
import datetime
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

%matplotlib inline

In [40]:
url = 'https://raw.githubusercontent.com/ngophuong/VNStock/main/excel_tcb.csv'
df = pd.read_csv(url, index_col = 0)
df['datetime']=pd.to_datetime(df['<DTYYYYMMDD>'],format='%Y%m%d')
df['Week_no']= df['datetime'].dt.week
df['Year_no']= df['datetime'].dt.year
df['Period'] = df['Year_no'].astype(str) + str("-") + df['Week_no'].astype(str)
df.head(3)

<ipython-input-40-7ef9f6d38ffb>:4: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.



,<DTYYYYMMDD>,<OpenFixed>,<HighFixed>,<LowFixed>,<CloseFixed>,<Volume>,<Open>,<High>,<Low>,<Close>,<VolumeDeal>,<VolumeFB>,<VolumeFS>,datetime,Week_no,Year_no,Period
<Ticker>,,,,,,,,,,,,,,,,,
TCB,20210415,41.95,41.95,40.9,40.9,14745100,41.95,41.95,40.9,40.9,1928000,0,0,2021-04-15,15,2021,2021-15
TCB,20210414,41.10,41.95,40.9,41.8,9753300,41.10,41.95,40.9,41.8,145000,0,0,2021-04-14,15,2021,2021-15
TCB,20210413,42.50,42.50,41.5,41.6,15104100,42.50,42.50,41.5,41.6,2915000,0,0,2021-04-13,15,2021,2021-15


In [41]:
df =df[['<Close>','datetime','<Volume>','Period','Year_no']]
df = df.head(1500)

In [42]:
#plot linechart for stock price
short_rolling = df['<Close>'].rolling(window=20).mean()

ema_long = df['<Close>'].ewm(span=50, adjust=False).mean()
ema_short = df['<Close>'].ewm(span=20, adjust=False).mean()
line_price = df['<Close>']
fig = go.Figure()
fig.update_layout(  autosize=False,   width=1200,    height=600)
#px.line(df_hpg, x='datetime', y='<Close>', labels={'x':'Date', 'y':'Price'})

fig.add_trace(go.Scatter(x=df.datetime, y=line_price, 
                        mode='lines', name='TCB'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_short, 
                        mode='lines', name='Span 20-days EMA'))

fig.add_trace(go.Scatter(x=df.datetime, y=ema_long, 
                        mode='lines', name='Span 50-days EMA'))



In [43]:
#px.violin(df_tips, y="tip", x="smoker", color="sex", box=True, points="all",
#          hover_data=df_tips.columns)

fig = go.Figure()
px.violin(df,x='Year_no',y='<Close>',box =True, points='all',
         hover_data=df.columns, title = 'VND Violin chart', width=900, height=600)

In [44]:
df.tail(3)

,<Close>,datetime,<Volume>,Period,Year_no
<Ticker>,,,,,
TCB,92.0,2018-06-06,1605480,2018-23,2018
TCB,96.0,2018-06-05,1357500,2018-23,2018
TCB,102.4,2018-06-04,2811840,2018-23,2018


In [45]:
#HDFC_df[‘Day_Perc_Change’] = HDFC_df[‘Adj Close’].pct_change()*100
df['Daily_return']=df['<Close>'].pct_change()*100
df.dropna(axis = 0, inplace = True)
df.head(3)

,<Close>,datetime,<Volume>,Period,Year_no,Daily_return
<Ticker>,,,,,,
TCB,41.80,2021-04-14,9753300,2021-15,2021,2.200489
TCB,41.60,2021-04-13,15104100,2021-15,2021,-0.478469
TCB,42.15,2021-04-12,14990200,2021-15,2021,1.322115


In [46]:
fig = go.Figure()
px.line(df, x='datetime', y='Daily_return', labels={'x':'Date', 'y':'Daily_Return_%'})

In [47]:
df['Daily_return'].describe()

count    719.000000
mean       0.263163
std        7.302750
min       -6.504065
25%       -0.961538
50%        0.000000
75%        0.852545
max      187.744227
Name: Daily_return, dtype: float64

In [48]:
fig = go.Figure()
px.histogram(df, x="Daily_return",histnorm='percent')
#ff.create_distplot(df, group_labels)

In [49]:
df['<Close>'].describe()

count    719.000000
mean      27.147983
std       13.753595
min       14.900000
25%       21.400000
50%       23.750000
75%       27.000000
max      109.000000
Name: <Close>, dtype: float64

In [50]:
dfig = go.Figure()
px.histogram(df.head(60), x="Daily_return",histnorm='percent')

In [51]:


df['Volumn'] = df['<Volume>']/1000000
Volumnshort_rolling = df['Volumn'].rolling(window=7).mean()

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.datetime, y=df['Daily_return'], 
                        mode='lines', name='Dailly_Retun'))
fig.add_trace(go.Scatter(x=df.datetime, y=df['Volumn'], 
                        mode='lines+markers', name='Volumn'))

fig.add_trace(go.Scatter(x=df.datetime, y=Volumnshort_rolling, 
                        mode='lines', name='Rolling_volumn'))

In [52]:
df['Volumn'].head(15).describe()

count    15.000000
mean     10.565947
std       2.601344
min       7.518800
25%       8.320950
50%      10.304100
75%      11.045700
max      15.104100
Name: Volumn, dtype: float64

In [53]:
df['Volumn'].head(10).describe()

count    10.000000
mean     10.842700
std       3.143301
min       7.518800
25%       8.082200
50%      10.389450
75%      13.984200
max      15.104100
Name: Volumn, dtype: float64